# Build a 2-hidden layers fully connected neural network with TensorFlow.
## This example is using some of TensorFlow highter-lever wrappers(tf.estimatore,tf.layers, tf.metrics, ...)

<img src="https://camo.githubusercontent.com/269f47b8185a2ca349ead57db511250553fd918b/687474703a2f2f63733233316e2e6769746875622e696f2f6173736574732f6e6e312f6e657572616c5f6e6574322e6a706567"
style="width:400px;height:200px;float:left">

## This example is using MNIST handwritten digits.

<img src="https://camo.githubusercontent.com/b06741b45df8ffe29c7de999ab2ec4ff6b2965ba/687474703a2f2f6e657572616c6e6574776f726b73616e64646565706c6561726e696e672e636f6d2f696d616765732f6d6e6973745f3130305f6469676974732e706e67"
style="width:200px;height:200px;float:mid">

## Importing the libraries

In [1]:
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Setting the parameter

In [2]:
learning_rate = 0.1
num_steps = 1000
batch_size = 128
display_step = 100

## Setting the network parameter 

In [3]:
n_hidden_1 = 256 # 1st layer number of neurous
n_hidden_2 = 256 # 2nd layer number of neurous
num_input = 784  # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0~9 digits)

## Define the input function for training

In [4]:
input_fn = tf.estimator.inputs.numpy_input_fn(
           x={'images': mnist.train.images}, y=mnist.train.labels,
           batch_size=batch_size, num_epochs=None, shuffle=True)

## Define the neural network

In [5]:
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurous
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

## Define the model function (following TF Estimator Template)

In [6]:
def model_fn(features, labels, mode):
    # Build the neural network
    logits = neural_net(features)
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return or train mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    # Define loss and opimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
              logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # The different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
                  mode=mode, predictions= pred_classes,
                  loss=loss_op, train_op = train_op,
                  eval_metric_ops={'accuracy': acc_op})
    return estim_specs

## Build the Estimator

In [7]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp0rlq38hp', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


## Train the model

In [9]:
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp0rlq38hp/model.ckpt.
INFO:tensorflow:loss = 2.312779, step = 1
INFO:tensorflow:global_step/sec: 275.941
INFO:tensorflow:loss = 0.3444283, step = 101 (0.363 sec)
INFO:tensorflow:global_step/sec: 267.548
INFO:tensorflow:loss = 0.423562, step = 201 (0.374 sec)
INFO:tensorflow:global_step/sec: 278.141
INFO:tensorflow:loss = 0.48579657, step = 301 (0.359 sec)
INFO:tensorflow:global_step/sec: 252.049
INFO:tensorflow:loss = 0.2219752, step = 401 (0.397 sec)
INFO:tensorflow:global_step/sec: 244.022
INFO:tensorflow:loss = 0.31634718, step = 501 (0.410 sec)
INFO:tensorflow:global_step/sec: 281.968
INFO:tensorflow:loss = 0.33601034, step = 601 (0.355 sec)
INFO:tensorflow:global_step/sec: 284.41
INFO:tensorflow:loss = 0.34160495, step = 701 (0.352 sec)
INFO:tensorflow:global_step/sec: 306.318
INFO:tensorflow:loss = 0.35427535, step = 801 (0.326 sec)
INFO:tensorflow:global_step/sec: 258.474
INFO:tensor

## Evaluate the Model
## Define the input function for evaluating

In [10]:
input_fn = tf.estimator.inputs.numpy_input_fn(
           x={'images': mnist.test.images}, y=mnist.test.labels,
           batch_size=batch_size, shuffle=False)

## Use the Estimator 'evaluate' method

In [11]:
model.evaluate(input_fn)

INFO:tensorflow:Starting evaluation at 2018-09-12-03:00:04
INFO:tensorflow:Restoring parameters from /tmp/tmp0rlq38hp/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-09-12-03:00:04
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9212, global_step = 1000, loss = 0.28740835


{'accuracy': 0.9212, 'loss': 0.28740835, 'global_step': 1000}

# Predict single images

In [12]:
n_images = 4

## Get images from test set

In [13]:
test_images = mnist.test.images[:n_images]

## Prepare the input data

In [14]:
input_fn = tf.estimator.inputs.numpy_input_fn(
           x={'images': test_images}, shuffle=False)

## Use the model to predict the images class

In [15]:
preds = list(model.predict(input_fn))

INFO:tensorflow:Restoring parameters from /tmp/tmp0rlq38hp/model.ckpt-1000


## Display

In [ ]:
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28,]))